<a href="https://colab.research.google.com/github/frankbass3/RareClassesPrediction/blob/main/STANFORD_Rarecalsses_IMAGE_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%ls /content/drive/MyDrive/Colab\ Notebooks/DATI-stanford-test-immagini/archive/

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/DATI-stanford-test-immagini/archive/

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

batch_size = 32
img_height = 224
img_width = 224

tds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

vds = tf.keras.preprocessing.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)




In [ ]:
# for x,y in train_list_ds:
#   print(y)
#   break
from IPython.display import Image, display
import matplotlib.pyplot as plt
for x,y in tds:
  print(x.shape)
  for xx in x:
    print(xx.shape)
    plt.imshow(xx.numpy().astype('uint8'))
    plt.show()
    break
  break

In [ ]:
total_x = []
total = []
for x,ba_y in tds:
  list_y = []
  for y in ba_y:
    if y in [1,2,3,4]:
      list_y.append(0)
    else:
      list_y.append(1)
  
  if ba_y.shape != 32:
    continue
  total.append(list_y)
  total_x.append(x)
  # print(x.shape)
  # print(ba_y.shape)

train_list_ds = tf.data.Dataset.from_tensor_slices((total_x, total))

total_x = []
total = []
for x,ba_y in vds:
  list_y = []
  for y in ba_y:
    if y in [1,2,3,4]:
      list_y.append(0)
    else:
      list_y.append(1)
  
  if ba_y.shape != 32:
    continue

  total.append(list_y)
  total_x.append(x)

val_ds = tf.data.Dataset.from_tensor_slices(( total_x, total))


In [ ]:
print(len(list(train_list_ds.as_numpy_iterator())))

In [ ]:
#dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
import tensorflow_hub as hub


module_selection = ("mobilenet_v2_100_224", 224)
handle_base, pixels = module_selection
do_fine_tuning = False
IMAGE_SIZE = (pixels, pixels)
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/classification/4".format(handle_base)


pretrainmodel = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, output_shape=[1001], trainable=do_fine_tuning),
    # tf.keras.layers.Dropout(rate=0.2),
    # tf.keras.layers.Dense(1280,
    #                       kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
pretrainmodel.build((None,)+IMAGE_SIZE+(3,))
pretrainmodel.summary()


In [ ]:
from tensorflow.python.keras.utils.vis_utils import plot_model
plot_model(pretrainmodel, show_shapes=True, show_layer_names=True)

In [ ]:
numberofpsudoclass = 1001 

def custom_loss(lam):
  def loss(y_true, y_pred):
    # Do your loss on your subset
      y_pred_n_aux, y_pred_aux, pseudo_lables  = y_pred[:, :1], y_pred[:, 1:numberofpsudoclass+1], y_pred[:, numberofpsudoclass+1:]
      cce1 = tf.keras.losses.CategoricalCrossentropy()
      cce2 = tf.keras.losses.CategoricalCrossentropy()

      l1 = cce1(y_true, y_pred_n_aux)
      #tf.print(pseudo_lables)

      l2 = cce2(pseudo_lables, y_pred_aux)
      #l1 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_pred_n_aux, logits=y_true)
      #l2 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_pred_aux,logits=pseudo_lables)
      return (l1 + lam*l2) 
  return loss

In [ ]:
import tensorflow_addons as tfa
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K

def custom_layer(pseudo_lables):
  #tf.print(input[0])
  col_indices = tf.cast(tf.argmax(pseudo_lables, axis=1), tf.int32)
  rows = tf.range(tf.shape(tf.argmax(pseudo_lables, axis=1))[0])
  sta = tf.stack([rows,col_indices], axis=1)
  # tf.print(tf.shape(pseudo_lables))
  # tf.print(tf.shape(rows))
  # tf.print(tf.shape(col_indices))
  # tf.print(tf.shape(sta))
  pred_max = tf.compat.v1.sparse_to_dense(sta,tf.shape(pseudo_lables),default_value=0.0,sparse_values=1.0)
  return pred_max

def custom_layer2(input):
  tf.print(input)
  return input

METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Accuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tfa.metrics.F1Score(name='f1', num_classes=2)
]



num_classes = 1
pseudo_lab_class = 1001

input = Input(shape=(224, 224,3), name="base_input")
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
# x = Block(filters=64, kernel_size=3, repetitions=2)(x)
# x = Block(filters=128, kernel_size=3, repetitions=2)(x)
# x = Block(filters=256, kernel_size=3, repetitions=3)(x)
# x = Block(filters=512, kernel_size=3, repetitions=3)(x)
# x = Block(filters=512, kernel_size=3, repetitions=3)(x)
x = tf.keras.layers.Conv2D(16, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)


y = tf.keras.layers.Dense(num_classes,activation='softmax')(x)
y2 = tf.keras.layers.Dense(pseudo_lab_class, activation='softmax')(x)

x2 = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
y_pseudo_lab = pretrainmodel(x2)
#y_pseudo_lab = tf.keras.layers.Lambda(custom_layer2)(y_pseudo_lab)
y_pseudo_lab = tf.keras.layers.Lambda(custom_layer)(y_pseudo_lab)

concat = tf.keras.layers.Concatenate()([y,y2,y_pseudo_lab])

archi = Model(inputs=input, outputs=concat)
archi.compile(loss=custom_loss(0.2), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics='accuracy')
#tb_callback = tf.keras.callbacks.TensorBoard('/content/', update_freq=1)

# tf.debugging.experimental.enable_dump_debug_info(
#    # logdir="/content/tfdbg2_logdir",
#     tensor_debug_mode="FULL_HEALTH",
#     circular_buffer_size=-1,
#     dump_root='/content/')

history = archi.fit(
  train_list_ds,
  validation_data=val_ds,
  epochs=9  #
   #callbacks=[tb_callback]
)



In [ ]:
import tensorflow_addons as tfa
import sys
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K


METRICS = [
      #tf.keras.metrics.TruePositives(name='tp'),
      #tf.keras.metrics.FalsePositives(name='fp'),
      #tf.keras.metrics.TrueNegatives(name='tn'),
      #tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.Accuracy(name='accuracy'),
      #tf.keras.metrics.Precision(name='precision'),
      #tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
      #tfa.metrics.F1Score(name='f1', num_classes=2)
]



pseudo_lab_class = 1001

input = Input(shape=(224, 224,3), name="base_input")
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3))(input)
x = tf.keras.layers.Conv2D(16, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, padding='same', activation=tf.nn.relu)(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

y = tf.keras.layers.Dense(1,activation='softmax')(x)

archi = Model(inputs=input, outputs=y)
archi.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=METRICS)

history = archi.fit(
  train_list_ds,
  validation_data=val_ds,
  epochs=15  #
   #callbacks=[tb_callback]
)



In [ ]:
plot_model(archi, show_shapes=True, show_layer_names=True)

In [ ]:
%load_ext tensorboard

%tensorboard --logdir /content/train

In [ ]:
tensorboard --logdir /content/